In [1]:
import pandas as pd
from openfisca_ceq.tools.survey_scenario import build_ceq_survey_scenario
from openfisca_ceq.tools.indirect_taxation.tax_benefit_system_indirect_taxation_completion import indirect_tax_by_country
from openfisca_ceq.tools.data import year_by_country
from openfisca_ceq.tools.data_ceq_correspondence import (
    ceq_input_by_harmonized_variable,
    ceq_intermediate_by_harmonized_variable,
    non_ceq_input_by_harmonized_variable,
    )

# Build country scenario

## Country selection

In [2]:
country = "cote_d_ivoire"

In [3]:
# Parameters
country = "mali"


In [4]:
year = year_by_country[country]
survey_scenario = build_ceq_survey_scenario(legislation_country = country, year = year)

# Aggegates check

## Income variables

### Disagregated income variables

In [5]:
ceq_by_harmonized_variable = dict()
ceq_by_harmonized_variable.update(ceq_input_by_harmonized_variable)
ceq_by_harmonized_variable.update(ceq_intermediate_by_harmonized_variable)
ceq_by_harmonized_variable.update(non_ceq_input_by_harmonized_variable)
ceq_by_harmonized_variable['salaire_brut'] = 'salaire_brut'
ceq_by_harmonized_variable

{'rev_i_autoconsommation': 'autoconsumption',
 'rev_i_autres': 'other_income',
 'rev_i_autres_transferts': 'gifts_sales_durables',
 'rev_i_loyers_imputes': 'imputed_rent',
 'rev_i_transferts_publics': 'direct_transfers',
 'rev_i_agricoles': 'revenu_agricole',
 'rev_i_autres_revenus_capital': 'autres_revenus_du_capital',
 'rev_i_independants_Ntaxe': 'revenu_informel_non_salarie',
 'rev_i_independants_taxe': 'revenu_non_salarie',
 'rev_i_independants': 'revenu_non_salarie_total',
 'rev_i_locatifs': 'revenu_locatif',
 'rev_i_pensions': 'pension_retraite',
 'rev_i_salaires_formels': 'salaire',
 'rev_i_salaires_informels': 'revenu_informel_salarie',
 'salaire_brut': 'salaire_brut'}

In [6]:
data = [
    (harmonized_variable, openfisca_variable, survey_scenario.compute_aggregate(openfisca_variable, period = year) / 1e9) 
    for harmonized_variable, openfisca_variable in ceq_by_harmonized_variable.items()
    ]

In [7]:
revenus = pd.DataFrame(data, columns = ["harmonized", "openfisca", "aggregate"])
revenus

,harmonized,openfisca,aggregate
0,rev_i_autoconsommation,autoconsumption,256.513891
1,rev_i_autres,other_income,282.008977
2,rev_i_autres_transferts,gifts_sales_durables,70.479250
3,rev_i_loyers_imputes,imputed_rent,150.021121
4,rev_i_transferts_publics,direct_transfers,0.000000
5,rev_i_agricoles,revenu_agricole,612.449204
6,rev_i_autres_revenus_capital,autres_revenus_du_capital,0.000000
7,rev_i_independants_Ntaxe,revenu_informel_non_salarie,511.791472
8,rev_i_independants_taxe,revenu_non_salarie,0.000000
9,rev_i_independants,revenu_non_salarie_total,511.791472


### Agregated income variables

#### CEQ agregated income variables

In [8]:
ceq_variables = ["market_income", "market_income_plus_pensions", "pensions", "contributions_pensions", "net_market_income", "direct_taxes", "other_contributions", "contributions_health", ]

In [9]:
ceq_aggregates = pd.DataFrame(
    index = ceq_variables,
    columns = ['aggregate'],
    data = [
        survey_scenario.compute_aggregate(ceq_variable, period = year) / 1e9 
        for ceq_variable in ceq_variables
        ]
    )
ceq_aggregates

,aggregate
market_income,2079.505036
market_income_plus_pensions,2079.505036
pensions,0.000000
contributions_pensions,0.000000
net_market_income,2079.505036
direct_taxes,0.000000
other_contributions,0.000000
contributions_health,0.000000


#### Direct check

In [10]:
revenu_components_summed_in_total = list(ceq_by_harmonized_variable.values()) 
revenu_components_summed_in_total.remove("revenu_non_salarie_total")
revenu_components_summed_in_total.remove("direct_transfers")
revenu_components_summed_in_total.remove("salaire")
revenu_components_summed_in_total += ["salaire_super_brut"]
total_revenu = sum(survey_scenario.compute_aggregate(revenu_component, period = survey_scenario.year) for revenu_component in revenu_components_summed_in_total) / 1e9
total_revenu

2079.5050359234588

In [11]:
# from numpy.testing import assert_almost_equal
# assert_almost_equal(ceq_aggregates.loc['market_income_plus_pensions'].values, total_revenu, decimal = 5) 

## Education variables

### Counts (in millions)

In [12]:
print((survey_scenario.compute_pivot_table(columns = ['eleve_enseignement_niveau'], index = ['eleve_enseignement_public'], aggfunc = 'count', period = survey_scenario.year) / 1e6).round(2))

eleve_enseignement_niveau  -1.0   0.0   1.0   2.0   3.0
eleve_enseignement_public                              
1.0                        12.4  0.11  1.77  1.27  0.12


In [13]:
variables = [
    'pre_school_person',
    'pre_school',
    'primary_education_person',
    'primary_education',
    'secondary_education_person',
    'secondary_education',
    'tertiary_education_person',
    'tertiary_education',
    'education_net_transfers',
    ]
print("In kind education tranfers")
for variable in variables:
    print(
        "{variable}: {aggregate} billions FCFA".format(
            variable = variable,
            aggregate = int(round(survey_scenario.compute_aggregate(variable, period = survey_scenario.year) / 1e9))
            )
        )

In kind education tranfers
pre_school_person: 0 billions FCFA
pre_school: 0 billions FCFA
primary_education_person: 119 billions FCFA
primary_education: 119 billions FCFA
secondary_education_person: 179 billions FCFA
secondary_education: 179 billions FCFA
tertiary_education_person: 16 billions FCFA
tertiary_education: 16 billions FCFA
education_net_transfers: 314 billions FCFA


## Tax variables

### Indirect tax variables

In [14]:
indirect_tax_variables = [
    variable 
    for tax in indirect_tax_by_country[country]
    for variable in survey_scenario.tax_benefit_system.variables.keys() 
    if tax in variable 
    ]
indirect_tax_variables

['depenses_ht_tva_exonere',
 'depenses_ht_tva_taux_normal',
 'depenses_ht_tva_taux_reduit',
 'tva_taux_normal',
 'tva_taux_reduit',
 'tva']

In [15]:
 pd.DataFrame(
    index = indirect_tax_variables,
    columns = ['aggregate'],
    data = [
        survey_scenario.compute_aggregate(variable, period = year) / 1e9 
        for variable in indirect_tax_variables
        ]
    )

,aggregate
depenses_ht_tva_exonere,1198.248234
depenses_ht_tva_taux_normal,1238.723146
depenses_ht_tva_taux_reduit,0.404737
tva_taux_normal,222.970175
tva_taux_reduit,0.020237
tva,222.990412


In [16]:
survey_scenario.compute_aggregate('consumption', period = year) / 1e9

2660.366454972495

In [17]:
survey_scenario.tax_benefit_system.variables.keys()

dict_keys(['household_weight', 'person_weight', 'age', 'date_naissance', 'marie', 'conjoint_a_des_revenus', 'salaire', 'salaire_brut', 'impot_brut', 'impot_traitement_salaire', 'nombre_enfants_a_charge', 'reduction_charge_famille', 'revenu_net_imposable', 'salaire_net_a_payer', 'accidents_du_travail', 'anpe', 'cotisations_employeur', 'cotisations_salariales', 'famille', 'retraite_employeur', 'retraite_salarie', 'salaire_imposable', 'salaire_super_brut', 'sante_employeur', 'sante_salarie', 'contributions_health', 'contributions_pensions', 'employee_contributions_health', 'employee_contributions_pensions', 'employee_other_contributions', 'employer_contributions_health', 'employer_contributions_pensions', 'employer_other_contributions', 'other_contributions', 'self_employed_contributions_health', 'self_employed_contributions_pensions', 'self_employed_other_contributions', 'agricultural_inputs_subsidies', 'electricity_subsidies', 'food_subsidies', 'fuel_subsidies', 'indirect_subsidies', 'c